In [47]:
from scipy.stats import norm
import numpy as np
import pandas as pd
from datetime import datetime
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import math
import random

#BINOMIAL MODEL WITHOUT DIVIDENDS
#n: number of periods in tree
#cp: +1.0 for calls, -1.0 for puts (enter float)
#am: 1 for American, 0 for European
#model: 1 CRR, 2 JR
def BINOMIAL_PRICE0(S0,K,T,sig,rf,n,cp,am=0,model=1):
    #DEFAULT model=1 (CRR)
    #BASIC SET UP
    h = T/n   #Length of each time period
    S = np.zeros(np.array([n+1,n+1])); S[0,0]=S0
    if model==1:  #If CRR, Model=1; If JR, Model=2
        u = math.exp(sig*math.sqrt(h))
        d = 1/u
        R = math.exp(rf*h); 
    if model==2:  
        u = math.exp((rf-0.5*sig**2)*h+sig*math.sqrt(h)); 
        d = math.exp((rf-0.5*sig**2)*h-sig*math.sqrt(h)); 
        R = math.exp(rf*h); 
    q = (R-d)/(u-d)
    
    #GENERATE STOCK TREE
    for j in range(0,n):
        S[0,j+1] = S[0,j]*u
        for i in range(0,j+1):
            S[i+1,j+1] = S[i,j]*d
            
    #OPTION VALUATION SEGMENT
    C = np.zeros(np.array([n+1,n+1]))
    C[:,n] = np.maximum(0.0,cp*(S[:,n]-K))
    for j in range(n,-1,-1):
        C[0:j,j-1] = (q*C[0:j,j] + (1-q)*C[1:j+1,j])/R
        if am==1:
            C[0:j,j-1] = np.maximum(C[0:j,j-1],cp*(S[0:j,j-1]-K))
    return C[0,0]


#BINOMIAL MODEL WITH CONTINUOUS DIVIDENDS
#n: number of periods in tree
#cp: +1.0 for calls, -1.0 for puts (enter float)
#am: 1 for American, 0 for European
#model: 1 CRR, 2 JR
def BINOMIAL_PRICE(S0,K,T,sig,rf,dv,n,cp,am=0,model=1):
    #DEFAULT model=1 (CRR)
    #BASIC SET UP
    h = T/n   #Length of each time period
    S = np.zeros(np.array([n+1,n+1])); S[0,0]=S0
    if model==1:  #If CRR, Model=1; If JR, Model=2
        u = math.exp(sig*math.sqrt(h)); d = 1/u; R = math.exp(rf*h); 
    if model==2:  
        u = math.exp((rf-0.5*sig**2)*h+sig*math.sqrt(h)); 
        d = math.exp((rf-0.5*sig**2)*h-sig*math.sqrt(h)); 
        R = math.exp(rf*h); 
    y = math.exp(dv*h)-1; 
    q=(R*math.exp(-dv*h)-d)/(u-d)
    for j in range(0,n):
        S[0,j+1] = S[0,j]*u
        for i in range(0,j+1):
            S[i+1,j+1] = S[i,j]*d
    divtree = S*y
    pvdiv = np.zeros(np.array([n+1,n+1]))
            
    #OPTION VALUATION SEGMENT
    C = np.zeros(np.array([n+1,n+1]))
    C[:,n] = np.maximum(0.0,cp*(S[:,n]-K))
    pvdiv[:,n] = divtree[:,n]
    for j in range(n,-1,-1):
        C[0:j,j-1] = (q*C[0:j,j] + (1-q)*C[1:j+1,j])/R
        pvdiv[0:j,j-1] = (q*pvdiv[0:j,j] + (1-q)*pvdiv[1:j+1,j])/R
        if am==1:
            C[0:j,j-1] = np.maximum(C[0:j,j-1],cp*(S[0:j,j-1]+pvdiv[0:j,j-1]-K))
        pvdiv[0:j,j-1] = pvdiv[0:j,j-1] + divtree[0:j,j-1]
    return C[0,0]



In [48]:
#Question Number 3
S = 100
K = 100
sig = 0.4
T = 1
rf = 0.05
n = 30

CallPriceE = BINOMIAL_PRICE0(S, K, T, sig, rf, n, 1.0, 0, 2)
print("European Call Price:", CallPriceE)

PutPriceE = BINOMIAL_PRICE0(S, K, T, sig, rf, n, -1.0, 0, 2)
print("European Put Price:", PutPriceE)

European Call Price: 18.05482856623461
European Put Price: 13.177771016306249


In [49]:
#Question Number 4
CallPriceA = BINOMIAL_PRICE0(S, K, T, sig, rf, n, 1.0, 1, 2)
print("American Call Price:", CallPriceA)
PutPriceA = BINOMIAL_PRICE0(S, K, T, sig, rf, n, -1.0, 1, 2)
print("American Put Price:", PutPriceA)

American Call Price: 18.05482856623461
American Put Price: 13.728935648119867


In [50]:
#Question Number 5
print(CallPriceE + K*math.exp(-rf*T))
print(PutPriceE + S)
print("Satisfies put-call parity")

113.17777101630602
113.17777101630625
Satisfies put-call parity
